In [1]:
import pandas as pd
import numpy as np
import time
# from pyspark.sql.functions import when

In [2]:
from pyspark import SparkContext, SparkConf

# Create a Spark configuration
conf = SparkConf().setAppName("MySparkApp")
# Create a SparkContext
sc = SparkContext(conf=conf)

# Set the log level to WARN
sc.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 16:48:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import sparknlp
# Start Spark Session
spark = sparknlp.start()

In [4]:
trainDataset = spark.read \
      .option("header", False) \
      .csv("train.csv")

In [5]:
header_names = ["label", "title", "text"]
trainDataset = trainDataset.toDF(*header_names)
# trainDataset = trainDataset.withColumn("label", when(trainDataset["label"] == 2, "positive").otherwise("negative"))

In [6]:
trainDataset.show(5)

+-----+--------------------+--------------------+
|label|               title|                text|
+-----+--------------------+--------------------+
|    2|Stuning even for ...|This sound track ...|
|    2|The best soundtra...|I'm reading a lot...|
|    2|            Amazing!|"This soundtrack ...|
|    2|Excellent Soundtrack|I truly like this...|
|    2|Remember, Pull Yo...|If you've played ...|
+-----+--------------------+--------------------+
only showing top 5 rows



In [7]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    UniversalSentenceEncoder,
    SentimentDLApproach
)

documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

useEmbeddings = UniversalSentenceEncoder.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment") \
    .setLabelColumn("label") \
    .setMaxEpochs(5) \
    .setEnableOutputLogs(True)

pipeline = Pipeline() \
    .setStages(
      [
        documentAssembler,
        useEmbeddings,
        sentimentdl
      ]
    )

pipelineModel = pipeline.fit(trainDataset)

!cat ~/annotator_logs/SentimentDLApproach_12faa854e3b3.log

TypeError: 'JavaPackage' object is not callable